In [ ]:
'''
Created on Feb 21, 2019

@author: Iswarya Vardhini
'''
import pandas as pd
from distributed import Client, progress
from hdfs import InsecureClient
import os
import projpk.masdriv as masdriv 
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

def immuprocess():
    immundf = masdriv.df
    print ("Data Cleaning process starts in Pneumonia Immunization Program")
    agesenwn = immundf[immundf._AGE65YR.notnull()]
    pnevacwn = agesenwn[agesenwn.PNEUVAC3.notnull()]
    pnemown = pnevacwn[pnevacwn._PNEUMO2.notnull()]
    pnemown = pnemown[pnemown._EDUCAG.notnull()]
    pnemown=pnemown[(pnemown._EDUCAG != 5) & (pnemown._EDUCAG != 6)& (pnemown._EDUCAG != 9)]
    pnemown = pnemown[(pnemown._AGE65YR != 3) & (pnemown._AGE65YR != 4)& (pnemown._AGE65YR != 5)& (pnemown._AGE65YR != 6)& (pnemown._AGE65YR != 7)& (pnemown._AGE65YR != 8)& (pnemown._AGE65YR != 9)& (pnemown._AGE65YR != 10)& (pnemown._AGE65YR != 11)& (pnemown._AGE65YR != 12)& (pnemown._AGE65YR != 13)]
    pnemown = pnemown[(pnemown.PNEUVAC3 != 7) & (pnemown.PNEUVAC3 != 9)] 
    pnemown = pnemown[(pnemown._PNEUMO2 != 9)]

    all3iwioutnulldf = pnemown

    '''print ("Printing size of im")
    i = immundf.isnull().apply(sum, axis=0) #prints number of Number of NAN values in a column
    #print (g)
    print (immundf.shape) #prints the number of columns and rows
    print ("After removing Na values in immun program")
    ai = all3iwioutnulldf.isnull().apply(sum, axis=0) #prints number of Number of NAN values in a column
    #print (ai)
    print(all3iwioutnulldf.shape) #prints the number of columns and rows

    print("Starting the analysis in Immun Program")
    sns.set() #Following commands are to find the distribution using Histogram
    plt.hist(all3iwioutnulldf['_AGE65YR'])
    plt.xlabel('Age Range')
    plt.ylabel('People count')
    plt.show()'''
    
    all3iwioutnulldf.to_csv('2017seniorpneumo_modified.csv',)
    
    groupcount = all3iwioutnulldf['PNEUVAC3'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3iwioutnulldf['_PNEUMO2'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3iwioutnulldf['_AGE65YR'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3iwioutnulldf['_EDUCAG'].value_counts() #Group wise count
    print (groupcount)
    
    
    limpy = all3iwioutnulldf['_EDUCAG'].corr(all3iwioutnulldf['_PNEUMO2'],method='pearson')# Corelation between variables. here Negative corelation
    print (limpy)
    print ("Anova between variables")
    anval = ols('_EDUCAG ~ _PNEUMO2', data=all3iwioutnulldf).fit()#One way anova
    anova_table = sm.stats.anova_lm(anval, typ=2)
    print (anova_table)
    
    '''sns.pairplot(all3iwioutnulldf[["_AGE65YR", "PNEUVAC3", "_PNEUMO2"]])# Pairplot Compares all 3 variables
    plt.show()'''
    
    model = ols('_PNEUMO2 ~ C(_EDUCAG)', all3iwioutnulldf).fit()
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
    res = sm.stats.anova_lm(model, typ= 2)
    print (res)
    
    print ("Linear regression model")
    x = all3iwioutnulldf["_EDUCAG"]
    y = all3iwioutnulldf["_PNEUMO2"]
    linearregmodel = sm.OLS(y, x).fit()
    predictions = linearregmodel.predict(x) # make the predictions by the model
    
    # Print out the statistics
    print (linearregmodel.summary())